In [1]:
from src.data_manager import DataManager

sheet_id = "***REMOVED***"

data_manager = DataManager(sheet_id, sheet_name="***REMOVED***")

df = data_manager.excel_to_dataframe("***REMOVED***")

In [2]:
from datetime import datetime, timedelta
import pandas as pd

from src.parse_excel_line import ParseExcelLine

line_parser = ParseExcelLine(df, datetime(2024, 1, 15))
line_parser.parse()

Index error list index out of range
Index error list index out of range


In [3]:
def select_week_from_date(df, date):
    col_from = pd.to_datetime(df["Semaine,du"], format="%Y-%m-%d", errors='coerce')
    col_to = pd.to_datetime(df["Semaine,au"], format="%Y-%m-%d", errors='coerce')

    selected_row = df.loc[(col_from <= date) & (date - timedelta(days=2) <= col_to)]
    
    return selected_row

In [12]:
line = select_week_from_date(df, datetime(2024, 1, 15))
line

,NaN,NaN,"Semaine,du","Semaine,au","Lundi,8h-9h","Lundi,9h-10h","Lundi,10h-11h","Lundi,11h-12h15","Lundi,14h-15h","Lundi,15h-16h",...,"Jeudi,16h15-17h15","Jeudi,17h15-18h15","Vendredi,8h-9h","Vendredi,9h-10h","Vendredi,10h-11h","Vendredi,11h-12h15","Vendredi,14h-15h","Vendredi,15h-16h","Vendredi,16h15-17h15","Vendredi,17h15-18h15"
22,temps 2,6.0,2024-01-15 00:00:00,2024-01-19 00:00:00,None,Java/ gradle\n (Julien Candela)\n 218,Java/ gradle\n (Julien Candela)\n 218,Java/ gradle\n (Julien Candela)\n 218,Java/ gradle\n (Julien Candela)\n 218,Java/ gradle\n (Julien Candela)\n 218,...,Equipe performante\n (Leonardo Santarelli)\n 210,None,None,Equipe performante\n (Leonardo Santarelli)\n 218,Equipe performante\n (Leonardo Santarelli)\n 218,Equipe performante\n (Leonardo Santarelli)\n 218,Equipe performante\n (Leonardo Santarelli)\n 218,Equipe performante\n (Leonardo Santarelli)\n 218,Equipe performante\n (Leonardo Santarelli)\n 218,Equipe performante\n (Leonardo Santarelli)\n 218


In [36]:
from src.course import Course

week_start_date = line["Semaine,du"].values[0]

jours_vers_chiffres = {
    "Lundi": 0,
    "Mardi": 1,
    "Mercredi": 2,
    "Jeudi": 3,
    "Vendredi": 4,
    "Samedi": 5,
    "Dimanche": 6
}

def get_date_from_day_of_week(week_start_date, day_of_week):
    try:
        return week_start_date + timedelta(days=jours_vers_chiffres[day_of_week])
    except KeyError:
        return None

# Transforms a string like "8h" or "8h30" into a time object
def translate_to_time(time_str):
    time_format = "%Hh%M"

    if time_str.endswith('h'):
        time_str = time_str + '00'

    translated_time = datetime.strptime(time_str, time_format).time()

    return translated_time

previous_course = Course()

def is_course_continuation(previous, current):
    if (previous.description == current.description 
            and (current.start - previous.end <= timedelta(minutes=30))):
        return True
    else:
        return False

course_list = []

for col in line:
    current_course = Course()
    current_course.description = line[col].values[0]
    
    # Splitting the header to get the day of the week and the time
    split = str(col).split(",")
    
    try:
        week_day = split[0]
        time = split[1].split("-")
        
        start_time = translate_to_time(time[0])
        end_time = translate_to_time(time[1])
        
        date = get_date_from_day_of_week(week_start_date,week_day)
        
        # Merge date and time
        current_course.start = datetime.combine(date, start_time)
        current_course.end = datetime.combine(date, end_time)
        
        # If the course is a continuation of the previous one, merge them
        if is_course_continuation(previous_course, current_course):
            current_course.start = previous_course.start
            print(current_course.start, current_course.end)
        # Base case: Add only if there is a course to memorize
        elif previous_course.description:
            print("Added " + str(previous_course))
            course_list.append(previous_course)
        
        previous_course = current_course
        
    except Exception as e:
        print(e)
    
course_list.append(previous_course)


list index out of range
list index out of range
time data 'du' does not match format '%Hh%M'
time data 'au' does not match format '%Hh%M'
2024-01-15 09:00:00 2024-01-15 11:00:00
2024-01-15 09:00:00 2024-01-15 12:15:00
Added Course: Java/ gradle
 (Julien Candela)
 218 (2024-01-15 09:00:00 - 2024-01-15 12:15:00)
2024-01-15 14:00:00 2024-01-15 16:00:00
2024-01-15 14:00:00 2024-01-15 17:15:00
Added Course: Java/ gradle
 (Julien Candela)
 218 (2024-01-15 14:00:00 - 2024-01-15 17:15:00)
2024-01-16 08:00:00 2024-01-16 10:00:00
2024-01-16 08:00:00 2024-01-16 11:15:00
2024-01-16 08:00:00 2024-01-16 12:15:00
Added Course: Langues (2024-01-16 08:00:00 - 2024-01-16 12:15:00)
2024-01-16 14:00:00 2024-01-16 16:00:00
2024-01-16 14:00:00 2024-01-16 17:15:00
2024-01-16 14:00:00 2024-01-16 18:15:00
Added Course: Projet 3A (2024-01-16 14:00:00 - 2024-01-16 18:15:00)
2024-01-17 09:00:00 2024-01-17 11:00:00
2024-01-17 09:00:00 2024-01-17 12:15:00
Added Course: Conception SI
 (Clément Sevy)
 228 (2024-01-17

In [37]:
[str(course_list[i]) for i in range(len(course_list))]

['Course: Java/ gradle\n (Julien Candela)\n 218 (2024-01-15 09:00:00 - 2024-01-15 12:15:00)',
 'Course: Java/ gradle\n (Julien Candela)\n 218 (2024-01-15 14:00:00 - 2024-01-15 17:15:00)',
 'Course: Langues (2024-01-16 08:00:00 - 2024-01-16 12:15:00)',
 'Course: Projet 3A (2024-01-16 14:00:00 - 2024-01-16 18:15:00)',
 'Course: Conception SI\n (Clément Sevy)\n 228 (2024-01-17 09:00:00 - 2024-01-17 12:15:00)',
 'Course: point POK sprint 2\n (FB-FM)\n 218 (2024-01-17 14:00:00 - 2024-01-17 16:00:00)',
 'Course: prez MON 2\n (FB-LP)\n 218 (2024-01-17 16:15:00 - 2024-01-17 18:15:00)',
 'Course: Equipe performante\n (Leonardo Santarelli)\n 210 (2024-01-18 09:00:00 - 2024-01-18 12:15:00)',
 'Course: Equipe performante\n (Leonardo Santarelli)\n 210 (2024-01-18 14:00:00 - 2024-01-18 17:15:00)',
 'Course: Equipe performante\n (Leonardo Santarelli)\n 218 (2024-01-19 09:00:00 - 2024-01-19 12:15:00)',
 'Course: Equipe performante\n (Leonardo Santarelli)\n 218 (2024-01-19 14:00:00 - 2024-01-19 18:15:0